0. Data and Libraries

0.1 Preprocessed Texts

In [32]:
from pandas import read_csv
df = read_csv("/content/EUSpeech_translated_preprocessed.csv")

0.2 Libraries (Especially Unsloth Download)

In [5]:
#capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
!pip install --upgrade xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 14.1 MB/s eta 0:00:00
  Attempting uninstall: xformers
    Found existing installation: xformers 0.0.29
    Uninstalling xformers-0.0.29:
      Successfully uninstalled xformers-0.0.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.3.19 requires tyro, which is not installed.
unsloth 2025.3.19 requires protobuf<4.0.0, but you have protobuf 5.29.4 which is incompatible.
unsloth 2025.3.19 requires trl!=0.15.0,!=0.9.0,!=0.9.1,!=0.9.2,!=0.9.3,<=0.15.2,>=0.7.9, but you have trl 0.16.0 which is incompatible.


1. Initialize Unsloth

In [9]:
!pip install --upgrade torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 5.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.3.19 requires tyro, which is not installed.
unsloth-zoo 2025.3.17 requires tyro, which is not installed.
xformers 0.0.29 requires torch==2.5.1, but you have torch 2.6.0 which is incompatible.
unsloth 2025.3.19 requires protobuf<4.0.0, but you have protobuf 5.29.4 which is incompatible.
unsloth 2025.3.19 requires trl!=0.15.0,!=0.9.0,!=0.9.1,!=0.9.2,!=

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2000 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model_Llama3.2_3B", # Pretrained unsloth model
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


2. Test Whether it Works

In [12]:
instruction = (
    "You are an expert political scientist specializing in discourse analysis. "
    "Your task is to analyze the following statement and determine whether it expresses any indication of solidarity towards another actor. "
    "For your response, return a JSON array with a single object containing two keys: 'Solidarity' and 'Justification'. "
    "The 'Solidarity' key should have the value 'Solidarity' if the statement shows support, or 'No Solidarity' if it does not. "
    "The 'Justification' key should provide a brief explanation (one or two sentences) citing specific phrases or elements that justify your label. \n\n"
    "For example, if the input is:\n"
    "\"I stand with you in these trying times, offering all my support,\" \n"
    "then the output should be:\n"
    "[\n"
    "  {\n"
    "    \"Solidarity\": \"Solidarity\",\n"
    "    \"Justification\": \"The statement explicitly offers support and solidarity through phrases like 'I stand with you' and 'offering all my support'.\"\n"
    "  }\n"
    "]\n\n"
    "If the input is:\n"
    "\"I am indifferent to your situation,\" \n"
    "then the output should be:\n"
    "[\n"
    "  {\n"
    "    \"Solidarity\": \"No Solidarity\",\n"
    "    \"Justification\": \"The statement does not express any form of support or solidarity, as indicated by the word 'indifferent'.\"\n"
    "  }\n"
    "]\n\n"
    "Now, analyze the following text:\n"
)
messages = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": "We as a Community of European States must hold together in times of crisis"}
]
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 300,
                   use_cache = True, temperature = 1.0, min_p = 0.1)

[
  {
    "Solidarity": "Solidarity",
    "Justification": "The statement uses the phrase must hold together', which implies unity and cooperation. Additionally, it references 'times of crisis', suggesting that the community is responding to a difficult situation."
  }
]<|eot_id|>


3. Prompts

3.1 Extract Solidarity Sentences

In [13]:
def code_solidarity_label(input_text, model, tokenizer):
    """
    This function instructs the model, an expert political scientist, to analyze a political statement
    and decide if it expresses any indication of solidarity towards another actor.
    The output is a JSON array with a single object containing two keys:
      - "Solidarity": "Solidarity" if the statement shows explicit support, assistance, or positive alignment towards another party;
                        "No Solidarity" otherwise.
      - "Justification": A brief explanation (one or two sentences) justifying the label, referring to specific phrases if applicable.

    Parameters:
        input_text (str): The text to analyze.
        model: The language model.
        tokenizer: The tokenizer for the model.

    Returns:
        str: The model's generated response as a structured JSON array.
    """

    # Define the instruction including examples
    instruction = (
        "You are an expert political scientist specializing in discourse analysis. "
        "Your task is to analyze the following statement and determine whether it expresses any indication of solidarity towards another actor. "
        "For your response, return a JSON array with a single object containing two keys: 'Solidarity' and 'Justification'. "
        "The 'Solidarity' key should have the value 'Solidarity' if the statement shows support, or 'No Solidarity' if it does not. "
        "The 'Justification' key should provide a brief explanation (one or two sentences) citing specific phrases or elements that justify your label. \n\n"
        "For example, if the input is:\n"
        "\"I stand with you in these trying times, offering all my support,\" \n"
        "then the output should be:\n"
        "[\n"
        "  {\n"
        "    \"Solidarity\": \"Solidarity\",\n"
        "    \"Justification\": \"The statement explicitly offers support and solidarity through phrases like 'I stand with you' and 'offering all my support'.\"\n"
        "  }\n"
        "]\n\n"
        "If the input is:\n"
        "\"I am indifferent to your situation,\" \n"
        "then the output should be:\n"
        "[\n"
        "  {\n"
        "    \"Solidarity\": \"No Solidarity\",\n"
        "    \"Justification\": \"The statement does not express any form of support or solidarity, as indicated by the word 'indifferent'.\"\n"
        "  }\n"
        "]\n\n"
        "Now, analyze the following text:\n"
    )

    # Build the prompt
    prompt = instruction

    # Set up the chat template for the tokenizer (e.g., "llama-3.1")
    tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

    # Enable 2x faster inference with FastLanguageModel
    FastLanguageModel.for_inference(model)

    # Prepare the messages list for chat-style generation.
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": input_text}
    ]

    # Apply the chat template to the messages.
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,  # Must add for generation
        return_tensors="pt",
    ).to("cuda")

    # Generate the output using the model
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,  # Adjust this value as needed
        use_cache=True,
        temperature=1.0,
        min_p=0.1
    )

    # Decode the generated tokens
    generated_text = tokenizer.batch_decode(outputs)[0].strip()

    return generated_text

3.2 Extract Solidarity Speech Acts

In [ ]:
def code_solidarity_speech_acts(input_text, model, tokenizer):
    """
    Function that instructs the model to analyze political text and extract solidarity-related speech acts,
    returning a structured JSON array with the identified information.

    Parameters:
        input_text (str): The political text to be analyzed.
        model: The language model.
        tokenizer: The tokenizer for the model.

    Returns:
        str: The coded response from the model as a structured JSON array.
    """

    # Optimized instruction for extracting solidarity-related speech acts with structured output
    instruction = (
        "You are an expert in political science and computational discourse analysis. "
        "Your task is to analyze the following political text to identify and code all instances of solidarity-related speech acts. "
        "A valid solidarity-related speech act must meet these criteria: "
        "1. Transitive Structure: The utterance must contain a transitive verb structure linking a subject (the provider, typically the speaker) to a direct object (the recipient). "
        "2. Positive Orientation: The subject expresses a positive orientation (they offer some form of assistance or help, even if verbal) toward the recipient, who is understood to be in a comparatively difficult situation. "
        "3. Illocutionary Type: The speech act must clearly fall into one of the following categories: "
        "   - Solidarity Expressives: Utterances that communicate emotional support or compassion without calling for specific action. "
        "   - Solidarity Directives: Utterances that demand, request, or call on others to act in support of the recipient. "
        "   - Solidarity Commissives: Utterances in which the speaker commits themselves to take future actions to support the recipient. "
        "For each solidarity-related speech act identified in the text, extract and return the following information: "
        "   - SpeechActType: Label the act as 'Expressive,' 'Directive,' or 'Commissive.' "
        "   - Provider: Identify the actor (usually the speaker) performing the solidarity action. "
        "   - Recipient: Identify the actor (direct object) who is in need and toward whom the positive orientation is directed. "
        "   - PropositionalContent: Summarize the core message or demand embedded in the speech act. "
        "If a sentence does not contain a solidarity-related speech act or lacks the required transitive structure, do not return anything. "
        "Return your analysis as a structured JSON array where each element represents an identified speech act with the keys "
        "'SpeechActType,' 'Provider,' 'Recipient,' and 'PropositionalContent.' "
        "Do not include any additional commentary or text beyond this structured output. "
        "For example, your output should look like this:\n"
        "[\n"
        "  {\n"
        "    \"SpeechActType\": \"\",\n"
        "    \"Provider\": \"\",\n"
        "    \"Recipient\": \"\",\n"
        "    \"PropositionalContent\": \"\"\n"
        "  }\n"
        "]\n"
        "Text to analyze:\n"
        f"{input_text}"
    )

    # Structuring the message with system and user roles
    messages = [
        {"role": "user", "content": instruction}
    ]

    # Creating the text-generation pipeline with a deterministic setting
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, temperature=0.9)
    output = pipe(messages)

    # Return the generated response containing the structured JSON analysis
    return output[0]['generated_text']


4. Run the LLM

4.1 Pre-Test

In [30]:
import re
result = code_solidarity_label("I stand with you in these trying times, offering all my support.", model, tokenizer)
print(result)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert political scientist specializing in discourse analysis. Your task is to analyze the following statement and determine whether it expresses any indication of solidarity towards another actor. For your response, return a JSON array with a single object containing two keys: 'Solidarity' and 'Justification'. The 'Solidarity' key should have the value 'Solidarity' if the statement shows support, or 'No Solidarity' if it does not. The 'Justification' key should provide a brief explanation (one or two sentences) citing specific phrases or elements that justify your label. 

For example, if the input is:
"I stand with you in these trying times, offering all my support," 
then the output should be:
[
  {
    "Solidarity": "Solidarity",
    "Justification": "The statement explicitly offers support and solidarity through phrases like 'I stand with you' and

4.2 JSON Parser

In [27]:
import json
import re

def get_solidarity_label(text, model, tokenizer):
    # Call your function that generates the JSON response.
    output = code_solidarity_label(text, model, tokenizer)

    # Use regex to extract all JSON blocks (non-greedy match)
    json_matches = re.findall(r'\[.*?\]', output, re.DOTALL)
    if json_matches:
        # Take the last JSON block, assuming it is the final answer.
        json_str = json_matches[-1]
    else:
        json_str = output  # Fallback if no JSON block is found

    try:
        data = json.loads(json_str)
        if isinstance(data, list) and len(data) > 0:
            return data[0]
        else:
            return {"Solidarity": None, "Justification": None}
    except json.JSONDecodeError:
        print("JSON decode error for text:", text)
        print("Output received:", output)
        return {"Solidarity": None, "Justification": None}

4.3 Run the Function

In [35]:
import pandas as pd
import json
from tqdm import tqdm

# Enable tqdm for pandas
tqdm.pandas()

# 1. Sample 1000 random rows from the original DataFrame
df_sampled = df.sample(n=1000, random_state=42)
df_remaining = df.drop(df_sampled.index)

# 2. Apply the model function with progress bar
results = df_sampled["context"].progress_apply(lambda x: get_solidarity_label(x, model, tokenizer))
results_df = pd.DataFrame(results.tolist())

# 3. Add the results to the sampled DataFrame
df_sampled = df_sampled.reset_index(drop=True)
df_sampled = pd.concat([df_sampled, results_df], axis=1)

# 4. For remaining rows, fill with "2" for both fields
df_remaining["Solidarity"] = "2"
df_remaining["Justification"] = "2"

# 5. Combine everything back together
df_final = pd.concat([df_sampled, df_remaining], axis=0).reset_index(drop=True)

# 6. Optional: save to CSV
df_final.to_csv("solidarity_labels_full.csv", index=False)

# 7. Preview
print(df_final.head())


 15%|█▌        | 153/1000 [09:43<1:07:16,  4.77s/it]

JSON decode error for text: It is very sad, but at the same time symptomatic and I think that will suffice as the entire commentary on the border, the wall, the barrier on the border, but I will also answer partially the questions of the Law and Justice and Confederation MPs. Of course, if we continue to govern, we will maintain the barrier on the border with Belarus. This is both part of our strategy to ensure internal security and part of our commitment to the voters, because about 10.5 million people in the referendum circled "no" in Answers to the question: Do you want the wall on the border with Belarus to be dismantled?
Output received: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert political scientist specializing in discourse analysis. Your task is to analyze the following statement and determine whether it expresses any indication of solidarity towards another actor. For your respon

 23%|██▎       | 228/1000 [14:44<42:48,  3.33s/it]

JSON decode error for text: Ukraine's European path was the central theme of my visit to Kyiv. Emmanuel Macron, Klaus Johannis, Mario Draghi and I made it clear in unison: Ukraine belongs to the European family. That is why I will do everything in my power at the European Council tomorrow and the day after to ensure that the entire EU says "yes" unanimously: 27 times "yes" to candidate status.
Output received: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert political scientist specializing in discourse analysis. Your task is to analyze the following statement and determine whether it expresses any indication of solidarity towards another actor. For your response, return a JSON array with a single object containing two keys: 'Solidarity' and 'Justification'. The 'Solidarity' key should have the value 'Solidarity' if the statement shows support, or 'No Solidarity' if it does not. The 'Justifica

 64%|██████▍   | 642/1000 [40:39<21:55,  3.68s/it]

...

JSON decode error for text: This also includes the ability and the will to resolve conflicts peacefully. In this spirit, NATO will continue its so-called "Intensified Dialogue" with Ukraine and Georgia. Since 1990, the network of NATO partnerships has been a key chapter in the successes of the "new" NATO.
Output received: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert political scientist specializing in discourse analysis. Your task is to analyze the following statement and determine whether it expresses any indication of solidarity towards another actor. For your response, return a JSON array with a single object containing two keys: 'Solidarity' and 'Justification'. The 'Solidarity' key should have the value 'Solidarity' if the statement shows support, or 'No Solidarity' if it does not. The 'Justification' key should provide a brief explanation (one or two sentences) citing specific phrase

 99%|█████████▉| 990/1000 [1:02:29<00:37,  3.80s/it]

JSON decode error for text: It has also disrupted supply chains, massively affected the global economy, and made it difficult for children and young people around the world to access education. The brutal war of aggression that Russia is waging against Ukraine is not only causing terrible suffering for the Ukrainian people. It has also led to record prices for grain and fertilizers, oil and gas - with serious consequences, especially in poorer countries.
Output received: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert political scientist specializing in discourse analysis. Your task is to analyze the following statement and determine whether it expresses any indication of solidarity towards another actor. For your response, return a JSON array with a single object containing two keys: 'Solidarity' and 'Justification'. The 'Solidarity' key should have the value 'Solidarity' if the statement sh

100%|██████████| 1000/1000 [1:03:13<00:00,  3.79s/it]

                                             context     Solidarity  \
0  You are thus part of a tradition of centuries-...  No Solidarity   
1  We are often asked whether this is so importan...     Solidarity   
2  We also have. We are opening soon, in a few da...  No Solidarity   
3  The picnic became a world event. But there was...  No Solidarity   
4  So, the entire industry and the entire energy ...  No Solidarity   

                                       Justification  
0  The statement does not express any form of sup...  
1  The statement explicitly states that Ukraine's...  
2  The statement does not express any form of sup...  
3  The statement does not express any form of sup...  
4  There is no indication of solidarity towards a...  
